In [ ]:
# !pip install import-ipynb

In [1]:
import pandas as pd
import numpy as np
import pyodbc
from scipy.stats import shapiro
from scipy.stats import kstest
from configparser import ConfigParser
from datetime import datetime
import statistics
# import import_ipynb
# import Entity_Risk

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
# clustered_data = pd.read_csv("/data/Synthetic_data_Clustered.csv")
data_ind_U = pd.read_csv(r"/data/peer_updated.csv")
#Uncomment to run on org
# data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

# data_ind = pd.read_csv("/data/Synthetic_data_Clustered_individual.csv")
# data_org = pd.read_csv("/data/Synthetic_data_Clustered_organization.csv")

In [4]:
# data_ind_U.head()

In [5]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [6]:

risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock) ", conn)

auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()

static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual') & (risk_lkp['RISK_TYPE'] != 'Auto High Rules ')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual') & (risk_lkp['RISK_TYPE'] != 'Auto High Rules ')]['RULE_NO'].to_list()

# non_stat_rules = tuple(non_stat_rules)

stat_rules = tuple(static_rules)

In [7]:
et_risk_df_per = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock) ", conn)

In [8]:
# et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(non_stat_rules), conn)

In [9]:
non_stat_rules

['P002001031',
 'P002001032',
 'P002001033',
 'P002001015',
 'P002001001',
 'P002001002',
 'P002001004',
 'P002001005',
 'P002001007',
 'P002001008',
 'P002001009',
 'P002001010',
 'P002001011',
 'P002001012',
 'P002001026',
 'P002001016',
 'P002001018',
 'P002001019',
 'P002001021',
 'P002001023',
 'P002001024',
 'P002001025',
 'P002001027',
 'P002001030',
 'P002001034',
 'P002003001',
 'P002003002',
 'P002003003',
 'P002003005',
 'P002003006',
 'P002003007',
 'P002003008',
 'P003001002',
 'P002001028',
 'P002001029']

In [10]:
# Read risk_person_summary_stg_rules for the static rules
if len(non_stat_rules) == 1:
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [11]:
non_stat_rules

('P002001031',
 'P002001032',
 'P002001033',
 'P002001015',
 'P002001001',
 'P002001002',
 'P002001004',
 'P002001005',
 'P002001007',
 'P002001008',
 'P002001009',
 'P002001010',
 'P002001011',
 'P002001012',
 'P002001026',
 'P002001016',
 'P002001018',
 'P002001019',
 'P002001021',
 'P002001023',
 'P002001024',
 'P002001025',
 'P002001027',
 'P002001030',
 'P002001034',
 'P002003001',
 'P002003002',
 'P002003003',
 'P002003005',
 'P002003006',
 'P002003007',
 'P002003008',
 'P003001002',
 'P002001028',
 'P002001029')

In [12]:
et_risk_df_per_rules

,PERSON_MASTER_ID,RULE_NO,RULE_VALUE,RISK_SCORE_STATIC,CREATE_DATE
0,1,P002001001,4,None,2023-02-21
1,1,P002001002,1,None,2023-02-21
2,1,P002001004,4,None,2023-02-21
3,1,P002001007,4,None,2023-02-21
4,1,P002001008,1,None,2023-02-21
...,...,...,...,...,...
2808,450,P002001005,1,None,2023-02-21
2809,450,P002001007,1,None,2023-02-21
2810,450,P002001008,1,None,2023-02-21
2811,450,P002001012,1,None,2023-02-21


In [13]:
auto_high_df = et_risk_df_per_rules[(et_risk_df_per_rules['RULE_NO'].isin(auto_high_rules)) & (et_risk_df_per_rules['RISK_SCORE_STATIC'] == 10)]

risk_per = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON]",conn)

auto_high_df = auto_high_df.groupby('PERSON_MASTER_ID')['RULE_NO'].unique().reset_index()

auto_high_df['RISK_RULE_LIST'] = auto_high_df['RULE_NO'].apply(lambda x:"_".join(x))

auto_high_df['RISK_SCORE'] = 10

In [14]:
auto_high_df

,PERSON_MASTER_ID,RULE_NO,RISK_RULE_LIST,RISK_SCORE


In [15]:
flaged_per_master_id = auto_high_df['PERSON_MASTER_ID'].to_list()
flaged_per_master_id

[]

In [16]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [17]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [18]:

#Create dataframe using pyodbc
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor1 = conn.cursor()
# cursor2 = conn.cursor()

summary_stg_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] with (nolock) ", conn)


In [19]:
summary_stg_df.head()

,PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FULL_NAME,FATHER_NAME,MOTHER_NAME,DATE_OF_BIRTH,GENDER,CBDT_INCOME_RANGE,TRAN_AMNT_CREDIT,TRAN_AMNT_DEBIT,PEER_GROUP_AMNT,PINCODE,LATITUDE,LONGITUDE,OCCUPATION_BUSINESS,ACCOUNT_COUNT,ADDRESS_COUNT,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,LEA_REQUESTS_COUNT,LNK_REPORT_COUNT,STR_REPORT_COUNT,STR_AMOUNT_DEBIT,STR_AMOUNT_CREDIT,CTR_REPORT_COUNT,CTR_AMOUNT_DEBIT,CTR_AMOUNT_CREDIT,NTR_REPORT_COUNT,NTR_AMOUNT_DEBIT,NTR_AMOUNT_CREDIT,PTR_REPORT_COUNT,PTR_AMOUNT_DEBIT,PTR_AMOUNT_CREDIT,CBWTR_REPORT_COUNT,CBWTR_AMOUNT_DEBIT,CBWTR_AMOUNT_CREDIT,CCR_REPORT_COUNT,CCR_AMOUNT_DEBIT,CCR_AMOUNT_CREDIT,CREATE_DATE,UPDATE_DATE
0,1,Government Sector_0,JDJPK5455L,None,64500000000000,Satyawan Hira Deshmukh,Hira,None,1993-03-14,Male,nan,2264360596.00000,566453535.00000,2264360596.00000,411026.00000,None,None,Government Sector,None,None,1.00000,nan,1.00000,None,None,None,4.00000,nan,565000000.00000,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
1,2,Government Sector_0,JDJPK5458L,None,65412564587515,Surya Mohammad Sami,None,None,1996-03-14,Male,nan,566453534.00000,nan,566453534.00000,411062.00000,None,None,Government Sector,None,None,nan,nan,nan,None,None,None,1.00000,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
2,3,Government Sector_0,JDJPK5459L,None,65412564587516,Satyawan Hira Deshmukh,None,None,1997-03-14,Female,nan,565000000.00000,566453535.00000,565000000.00000,411062.00000,None,None,Government Sector,None,None,nan,nan,nan,None,None,None,1.00000,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
3,4,Government Sector_0,JDJPK5460L,None,65412564587517,Satyawan Hira Deshmukh,None,None,1998-03-14,Female,nan,nan,566453536.00000,566453536.00000,411062.00000,None,None,Government Sector,None,None,nan,nan,nan,None,None,None,1.00000,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None
4,5,Government Sector_0,JDJPK5461L,None,65412564587518,Satyawan Hira Deshmukh,None,None,1999-03-14,Female,nan,nan,566453537.00000,566453537.00000,411062.00000,None,None,Government Sector,None,None,1.00000,nan,nan,None,None,None,1.00000,nan,nan,nan,None,nan,nan,nan,nan,None,None,None,nan,None,None,None,None,None,None,None


In [20]:
# summary_stg_df[summary_stg_df['PERSON_MASTER_ID'] == 1]['PEER_GROUP_ID']

In [21]:
temp = pd.merge(auto_high_df,summary_stg_df , on = 'PERSON_MASTER_ID',how = 'inner')

In [22]:
# temp.rename(columns = {'PEER_GROUP_ID_y':'PEER_GROUP_ID'}, inplace = True)

In [23]:
temp

,RULE_NO,RISK_RULE_LIST,RISK_SCORE,PERSON_MASTER_ID,PEER_GROUP_ID,PAN,DIN,CKYC_NUMBER,FULL_NAME,FATHER_NAME,MOTHER_NAME,DATE_OF_BIRTH,GENDER,CBDT_INCOME_RANGE,TRAN_AMNT_CREDIT,TRAN_AMNT_DEBIT,PEER_GROUP_AMNT,PINCODE,LATITUDE,LONGITUDE,OCCUPATION_BUSINESS,ACCOUNT_COUNT,ADDRESS_COUNT,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,LEA_REQUESTS_COUNT,LNK_REPORT_COUNT,STR_REPORT_COUNT,STR_AMOUNT_DEBIT,STR_AMOUNT_CREDIT,CTR_REPORT_COUNT,CTR_AMOUNT_DEBIT,CTR_AMOUNT_CREDIT,NTR_REPORT_COUNT,NTR_AMOUNT_DEBIT,NTR_AMOUNT_CREDIT,PTR_REPORT_COUNT,PTR_AMOUNT_DEBIT,PTR_AMOUNT_CREDIT,CBWTR_REPORT_COUNT,CBWTR_AMOUNT_DEBIT,CBWTR_AMOUNT_CREDIT,CCR_REPORT_COUNT,CCR_AMOUNT_DEBIT,CCR_AMOUNT_CREDIT,CREATE_DATE,UPDATE_DATE


In [24]:
# Adding the flaged person in auto high rules to risk person table

for ind,row in temp.iterrows():
    conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON] (PERSON_MASTER_ID, PEER_GROUP_ID, RISK_SCORE,RISK_RULE_LIST)
    values (?,?,?,?)''',(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],10,row['RISK_RULE_LIST']))

conn.commit()
conn.close()


In [25]:
et_risk_df_per_rules = et_risk_df_per_rules[~et_risk_df_per_rules['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [26]:
et_risk_df_per_rules

,PERSON_MASTER_ID,RULE_NO,RULE_VALUE,RISK_SCORE_STATIC,CREATE_DATE
0,1,P002001001,4,None,2023-02-21
1,1,P002001002,1,None,2023-02-21
2,1,P002001004,4,None,2023-02-21
3,1,P002001007,4,None,2023-02-21
4,1,P002001008,1,None,2023-02-21
...,...,...,...,...,...
2808,450,P002001005,1,None,2023-02-21
2809,450,P002001007,1,None,2023-02-21
2810,450,P002001008,1,None,2023-02-21
2811,450,P002001012,1,None,2023-02-21


In [27]:
et_risk_df_per_rules.drop_duplicates(inplace=True)

In [28]:
# et_risk_df_per_rules[et_risk_df_per_rules['PERSON_MASTER_ID'] == 76400]

In [29]:
# t = et_risk_df_per_rules.groupby(['PERSON_MASTER_ID','RULE_NO']).size().reset_index()
# t[t[0]>1]

In [30]:
#changing vertical format to horizontal
pivot_per_df = et_risk_df_per_rules
pivot_per_df = pivot_per_df.pivot(index='PERSON_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
pivot_per_df.columns = [col[1] for col in pivot_per_df.columns.values]
pivot_per_df = pivot_per_df.reset_index()
cols = pivot_per_df.columns.to_list()
cols.append('CREATE_DATE')
pivot_per_df = pivot_per_df.merge(et_risk_df_per_rules,on = 'PERSON_MASTER_ID')[cols]



# pivot_org_df = et_risk_df_org_rules
# pivot_org_df = pivot_org_df.pivot(index='ORGANIZATION_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
# pivot_org_df.columns = [col[1] for col in pivot_org_df.columns.values]
# pivot_org_df = pivot_org_df.reset_index()
# cols = pivot_org_df.columns.to_list()
# cols.append('CREATE_DATE')
# pivot_org_df = pivot_org_df.merge(et_risk_df_org_rules,on = 'ORGANIZATION_MASTER_ID')[cols]

In [31]:
pivot_per_df

,PERSON_MASTER_ID,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE
0,1,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
1,1,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2,1,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
3,1,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
4,1,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808,450,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2809,450,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2810,450,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2811,450,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21


In [32]:
data_ind_U = data_ind_U.merge(pivot_per_df,how = 'left',on = 'PERSON_MASTER_ID')
# et_risk_df_org = et_risk_df_org.merge(pivot_org_df,how='left',on = 'ORGANIZATION_MASTER_ID')
data_ind_U.head()

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
1,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
3,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
4,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21


In [33]:
data_ind_U = data_ind_U[~data_ind_U['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [34]:
data_ind_U.head()

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
1,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
2,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
3,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21
4,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21


In [35]:
# for rule in list(set(non_stat_rules) - set(auto_high_rules)):
# #     print(rule)
# #     data_ind_U[rule].fillna(value = 0,inplace = True)
#     data_ind_U[rule] = data_ind_U[rule].astype('int64') 

In [36]:
data_ind_U.dtypes

PERSON_MASTER_ID               int64
PEER_GROUP_AMNT              float64
PEER_GROUP_AMNT_Scaled       float64
PEER_GROUP_ID                 object
Cluster_labels                 int64
OCCUPATION_BUSINESS_CLEAN     object
P002001001                    object
P002001002                    object
P002001004                    object
P002001005                    object
P002001007                    object
P002001008                    object
P002001009                    object
P002001010                    object
P002001011                    object
P002001012                    object
P002001015                    object
P002001018                    object
P002001021                    object
P002001024                    object
P002001025                    object
P002001027                    object
P002001028                    object
P002001030                    object
P002001031                    object
P002001033                    object
P002001034                    object
P

In [37]:
# reqd_cols_U = [col for col in data_ind_U.columns if 'nbr' in col.lower()]
reqd_cols_per = list(set(non_stat_rules) - set(auto_high_rules)) #doing this to remove auto high rule from nan_stat_rule_list
#Uncomment to run on org
# reqd_cols_org = ['O002001001','O002001018','O002001017','O002001024', 'O002001025', 'O002001026']



# reqd_cols_U.append('TRAN_AMNT')
# reqd_cols_U

# reqd_cols_U
# reqd_cols = ['Transaction_Amount','Nbr_of_CTR_reports','Nbr_of_STR_Reports','Nbr_of_NTR_reports','WL_Count','Nbr_of_email_ids','Nbr_of_phones']

# Threshold Calculation with Outliers

In [38]:
# def compute_outlier(data, col):
# #     inf = float('inf')
# #     print("Hello",len(data[col]),type(data[col]))
# #     if len(data[col]) >= 30:
#         #if there are more than 30 data points, use z-score approach
    
#     try:    
#         d = data[col].dropna().astype('int64')
#         if len(d) == 0:
# #             return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
#             return None,None,None,None,None,None,None,None,None,None
#         else:
        
            
#             median = np.median(d)
#             # change it to median
#             std = np.std(d)
#     #         print(d,col,"****************")
#             one = round(median+std)
#             two = round(median+2*std)
#             three = round(median+3*std)
#             four = round(median+4*std)
#             five = round(median+5*std)
#             six = round(median+6*std)
#             seven = round(median+7*std)
#             eight = round(median+8*std)
#             nine =round(median+9*std)
#             ten = round(median+10*std)
#             return one,two,three,four,five,six,seven,eight,nine,ten        
#     except KeyError:
# #         return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
#         return None,None,None,None,None,None,None,None,None,None
# #         pass
# #         low = 0
# #         mid = mean
# #         high = np.max(data[col])
# #         low_inc = (mid-low)/5
# #         high_inc = (high-mid)/5
# #         one = low
# #         two = low+low_inc
# #         three = low+2*low_inc
# #         four = low+3*low_inc
# #         five = low+4*low_inc
# #         six = mid
# #         seven = mid+high_inc
# #         eight = mid+2*high_inc
# #         nine = mid+3*high_inc
# #         ten = mid+4*high_inc
    

# Threshold Calculations without Outliers

In [109]:
from scipy import stats as s

In [119]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right

In [120]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    return data,outliers_right
    
    

In [121]:
# def thresh_cal(d,outliers_right):
#     print("thresh")
#     median = np.median(d)
#     mean = np.mean(d)
#     std = np.std(d)
#     mode = int(s.mode(d)[0])
#     print("DATA = ",d,median,std,"********************************")
#     one = round(median+std)
#     two = round(median+2*std)
#     three = round(median+3*std)
#     four = round(median+4*std)
#     five = round(median+5*std)
#     six = round(median+6*std)
#     seven = round(median+7*std)
#     eight = round(median+8*std)
#     nine =round(median+9*std)
#     ten = round(median+10*std)
#     return one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right
    
    

In [7]:
def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])
    print("DATA = ",d,median,std,"********************************")
    
    if std >= 1:
        one = round(median+std)
        two = round(median+2*std)
        three = round(median+3*std)
        four = round(median+4*std)
        five = round(median+5*std)
        six = round(median+6*std)
        seven = round(median+7*std)
        eight = round(median+8*std)
        nine =round(median+9*std)
        ten = round(median+10*std)

    else:
        one = np.ceil(median+std)
        two = np.ceil(one+std)
        three = np.ceil(two+std)
        four = np.ceil(three+std)
        five = np.ceil(four+std)
        six = np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =np.ceil(eight+std)
        ten = np.ceil(nine+std)

    return one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right

In [18]:

# if std >= 1:
#     one = round(median+std)
#     two = round(median+2*std)
#     three = round(median+3*std)
#     four = round(median+4*std)
#     five = round(median+5*std)
#     six = round(median+6*std)
#     seven = round(median+7*std)
#     eight = round(median+8*std)
#     nine =round(median+9*std)
#     ten = round(median+10*std)
    
# else:
#     one = np.ceil(median+std)
#     two = np.ceil(one+std)
#     three = np.ceil(two+std)
#     four = np.ceil(three+std)
#     five = np.ceil(four+std)
#     six = np.ceil(five+std)
#     seven = np.ceil(six+std)
#     eight = np.ceil(seven+std)
#     nine =np.ceil(eight+std)
#     ten = np.ceil(nine+std)

In [19]:
# print(one,two,three,four,five,six,seven,eight,nine,ten)

2 3 4 5 6 7 8 9 10


In [ ]:
# def thresh_cal(d,outliers_right):
#     print("thresh")
#     median = np.median(d)
#     mean = np.mean(d)
#     std = np.std(d)
#     mode = int(s.mode(d)[0])
#     print("DATA = ",d,median,std,"********************************")
#     one = round(median+std)
    
#     if one == round(median+2*std):
#         two = np.ceil(one+std)
#     else:
#         two = round(median+2*std)
        
#     if two <= (median+3*std):
#         three = np.ceil(two+std)
#     else:
#         three = 
        
#         one = round(median+std)
#         if(one == round(median+std)):
#             counter = median
#         else:
#             counter = median
#         for i in range(2,11):
#             if(i ==  2):
#                 if(one == round(counter+i*std)):
#                     two = np.ceil(counter+i*std)
#                     counter = two
#                 else:
#                     two = round(counter+i*std)
#                     counter = two
#             if(i == 3):
#                 if(two == round(counter+i*std)):
#                     three = np.ceil(two+std)
#                     counter = three
#                 else:
#                     three = round(counter+i*std)
#                     counter = three 
#             if(i == 4):
#                 if(three == (counter+i*std)):
#                     four = np.ceil(+std)
#                     counter = four
#                 else:
#                     four = round(counter+i*std)
#                     counter = four 
        
        
    
#   std = 0.3, 1

# 1 - 0
# 2 - 1
#     if std >= 1:

#     one = round(median+std)    
#     two = round(median+2*std)
#     three = round(median+3*std)
#     four = round(median+4*std)
#     five = round(median+5*std)
#     six = round(median+6*std)
#     seven = round(median+7*std)
#     eight = round(median+8*std)
#     nine =round(median+9*std)
#     ten = round(median+10*std)
#     return one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right

In [122]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    print(d)
    data = [*set(d)]
    return compute_IQR(d,data)

In [123]:


def compute_outlier(data,col):
    try:
#         print("data = ",data)
        d = data[col].dropna().astype('float64')
        d = d.astype('int64')
#         print("d = ",d)
    
        if (len(d) == 0):
            print("Not applicable")
            return None,None,None,None,None,None,None,None,None,None
          

        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
            
        
        
        
        else:
            
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)   
                    
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
    
            
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None) 
    
    return (None,None,None,None,None,None,None,None,None,None)

In [124]:
class Thresholds():
    def __init__(self,df,reqd_cols):
        self.data = df
        self.reqd_cols_U = reqd_cols
    def get_thresholds(self):
        # Python3 dict to contain threshold calculations-
        # Python3 lists to contain threshold values for desired columns-       
        self.class_dict = {}
        for col in self.reqd_cols_U:
            self.class_dict[col] = []
        
        self.class_dict['OCCUPATION_BUSINESS_CLEAN'] = []
        self.class_dict['Cluster_labels'] = []

        for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             if occ == 'Others':
                for clabel in self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ, 'Cluster_labels'].unique():

                    for col in self.reqd_cols_U:
                        print(f"Profession: {occ}, Cluster = {clabel},"
                        f" col = {col}; outlier"
                        f" = {compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col)}"
                        )
                        # profession.append(occ)
                        # profession.append(clabel)
                        self.class_dict[col].append(compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col))

                    self.class_dict['OCCUPATION_BUSINESS_CLEAN'].append(occ)
                    self.class_dict['Cluster_labels'].append(clabel)
#                 print(self.class_dict)
#                 break
#         self.class_dict['Type_of_profession'] = []
#         self.class_dict['Cluster_labels'] = []
                        
#         for occ in self.data['Type_of_profession'].unique():
#             for clabel in self.data.loc[self.data['Type_of_profession'] == occ, 'Cluster_labels'].unique():
#                 # print(f"{occ}, {clabel}")
#                 self.class_dict['Type_of_profession'].append(occ)
#                 self.class_dict['Cluster_labels'].append(clabel)
        
        self.threshold_data = pd.DataFrame(self.class_dict)
        return self.threshold_data
#         return self.class_dict

In [125]:
data_ind_U

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE,P002001016,P002001029,P002001019,P003001002,P002001032,P002001023,P002003007,P002001026
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
11,2,566453534.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
21,3,565000000.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
31,4,566453536.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
41,5,566453537.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,447,30000025.00000,0.00001,Private Sector_0,0,Private Sector,1,1,1,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2797,448,0.00000,0.00000,Private Sector_0,0,Private Sector,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2799,449,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2806,450,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan


In [126]:
data_ind_U.drop_duplicates(inplace = True)

In [127]:
ind_U = Thresholds(data_ind_U,reqd_cols_per)
ind_data_thresholds_U = ind_U.get_thresholds()

#Uncomment to run on org
# org_U = Thresholds(data_org_U,reqd_cols_org)
# org_data_thresholds_U = org_U.get_thresholds()


Not applicable
Profession: Government Sector, Cluster = 0, col = P002001031; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicable
Not applicable
Profession: Government Sector, Cluster = 0, col = P002001032; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicable
not_normal_outliers_removal
0       4
11      1
21      1
31      1
41      1
       ..
2550    1
2563    1
2589    1
2615    1
2653    2
Name: P002001033, Length: 101, dtype: int64
compute_IQR
outliers_right 7.0
thresh
DATA =  [4, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 2, 2, 3, 2, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2] 1.0 0.5936328831862332 ********************************
Profession: Government Sector, Cluster = 0, col = P002001033; outlier = (2, 2, 3, 3, 4, 5

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "



501     1
1385    2
1404    1
1413    1
1422    1
1463    1
1690    1
2383    1
2404    1
Name: P002001005, dtype: int64
compute_IQR
outliers_right 2.5
thresh
DATA =  [1, 2, 1, 1, 1, 1, 1, 1, 1] 1.0 0.3142696805273545 ********************************
Profession: OTHERS_CHANGED, Cluster = 0, col = P002001005; outlier = (1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111112, 1.0, 1, 0.3142696805273545, 2.5)
not_normal_outliers_removal
501     1
1385    2
1404    1
1413    1
1422    1
1463    1
1690    1
2383    1
2404    1
Name: P002001005, dtype: int64
compute_IQR
outliers_right 2.5
thresh
DATA =  [1, 2, 1, 1, 1, 1, 1, 1, 1] 1.0 0.3142696805273545 ********************************
not_normal_outliers_removal
52      1
501     1
847     1
854     1
1302    1
1308    1
1385    2
1404    1
1413    1
1422    1
1678    1
1684    1
1690    1
2337    1
2358    1
2365    1
2383    1
2404    1
2665    1
Name: P002001007, dtype: int64
compute_IQR
outliers_right 2.5
thresh
DATA =  [1, 1, 1, 1, 1, 1, 2,

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


outliers_right 14.5
thresh
DATA =  [1, 2, 1, 1, 2, 7, 14, 5, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1] 2.0 2.8722813232690143 ********************************
not_normal_outliers_removal
325      2
429      2
442      7
454     14
940      2
952      2
964      2
977      2
989      2
1001     2
1013     2
1284     2
1439     1
1519     1
1529     1
2414     1
Name: P002001025, dtype: int64
compute_IQR
outliers_right 19.25
thresh
DATA =  [2, 2, 7, 14, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1] 2.0 3.1862742741327215 ********************************
Profession: OTHERS_CHANGED, Cluster = 9, col = P002001025; outlier = (5, 8, 12, 15, 18, 21, 24, 27, 31, 34, 2.8125, 2.0, 2, 3.1862742741327215, 19.25)
not_normal_outliers_removal
325      2
429      2
442      7
454     14
940      2
952      2
964      2
977      2
989      2
1001     2
1013     2
1284     2
1439     1
1519     1
1529     1
2414     1
Name: P002001025, dtype: int64
compute_IQR
outliers_right 19.25
thresh
DATA =  [2, 2, 7, 14, 2, 2

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for sh

DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.0 ********************************
normal_outliers_removal
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.0 ********************************
Profession: OTHERS_CHANGED, Cluster = 8, col = P002001007; outlier = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
normal_outliers_removal
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.0 ********************************
normal_outliers_removal
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.0 ********************************
Profession: OTHERS_CHANGED, Cluster = 8, col = P002001008; outlier = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 1.0)
normal_outliers_removal
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.0 ********************************
Not applicable
thresh
DATA =  791    1
Name: P002001009, dtype: int64 1.0 0.0 ********************************
Profession: OTHERS_CHANGED, Cluster = 8, col

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for sh

Not applicable
Not applicable
Profession: Others, Cluster = 0, col = P002003007; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicable
Not applicable
Profession: Others, Cluster = 0, col = P002003008; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicable
Not applicable
Profession: Others, Cluster = 0, col = P003001002; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicable
Not applicable
thresh
DATA =  659     1
1578    1
1584    1
Name: P002001028, dtype: int64 1.0 0.0 ********************************
Profession: Others, Cluster = 0, col = P002001028; outlier = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0.0, 'NA')
Not applicable
thresh
DATA =  659     1
1578    1
1584    1
Name: P002001028, dtype: int64 1.0 0.0 ********************************
Not applicable
Profession: Others, Cluster = 0, col = P002001029; outlier = (None, None, None, None, None, None, None, None, None, None)
Not applicab

In [129]:
ind_data_thresholds_U.head()

,P002001031,P002001032,P002001033,P002001015,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001026,P002001016,P002001018,P002001019,P002001021,P002001023,P002001024,P002001025,P002001027,P002001030,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003007,P002003008,P003001002,P002001028,P002001029,OCCUPATION_BUSINESS_CLEAN,Cluster_labels
0,"(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1190476190476...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.15, 1.0, 1, 0...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1076923076923...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1414141414141...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1204819277108...","(2, 3, 3, 4, 5, 6, 6, 7, 8, 9, 1.2923076923076...","(1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 1.1666666666666...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 1.1060606060606...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 1.234375, 1.0, ...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 1.26, 1.0, 1, 0...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(-74, -74, -74, -74, -74, -74, -74, -74, -74, ...","(23710, 23710, 23710, 23710, 23710, 23710, 237...","(26837, 26837, 26837, 26837, 26837, 26837, 268...","(-80, -80, -80, -80, -80, -80, -80, -80, -80, ...","(64351, 64351, 64351, 64351, 64351, 64351, 643...","(None, None, None, None, None, None, None, Non...","(-22, -22, -22, -22, -22, -22, -22, -22, -22, ...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 1.1034482758620...","(None, None, None, None, None, None, None, Non...",Government Sector,0
1,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1.0526315789473...","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.125, 1.0, 1, ...","(2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 1.571428571428...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1.1111111111111...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 1.1333333333333...","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 1.1, 1.0, 1, 0....","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1.0, 1, 0....","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None, None, None, None, None, None, Non...","(None, None,

In [130]:
# ind_data_thresholds_U[(ind_data_thresholds_U['OCCUPATION_BUSINESS_CLEAN'] == 'Business') & (data_ind_U['Cluster_labels'] == 1)]


In [131]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [132]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [133]:
#Create dataframe using pyodbc
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# data_wt = cursor.execute("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]").fetchall()

In [134]:
weight_sql = pd.read_sql_query("SELECT * FROM [Fincore_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)

In [135]:
weight_table_ind_U = ind_data_thresholds_U.copy()
#uncomment to run for org
# weight_table_org_U = org_data_thresholds_U.copy()

# weight_table_ind = ind_data_thresholds.copy()
# weight_table_org = org_data_thresholds.copy()

In [136]:
for col in ind_data_thresholds_U.columns:
    if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
        pass
    else:
        weight_table_ind_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]

#uncomment to run for org
# for col in org_data_thresholds_U.columns:
#     if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
#         pass
#     else:
#         weight_table_org_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]



In [137]:
weight_table_ind_U

,P002001031,P002001032,P002001033,P002001015,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001026,P002001016,P002001018,P002001019,P002001021,P002001023,P002001024,P002001025,P002001027,P002001030,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003007,P002003008,P003001002,P002001028,P002001029,OCCUPATION_BUSINESS_CLEAN,Cluster_labels
0,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,Government Sector,0
1,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,0
2,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,9
3,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,1
4,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,8
5,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,2
6,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,OTHERS_CHANGED,5
7,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,Others,0
8,1,5,3,3,6,6,8,8,6,5,4,6,6,5,5,5,6,6,6,8,5,5,3,3,1,7,7,5,7,8,7,7,8,1,5,Private Sector,0


In [138]:
data_ind_U

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE,P002001016,P002001029,P002001019,P003001002,P002001032,P002001023,P002003007,P002001026
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
11,2,566453534.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
21,3,565000000.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
31,4,566453536.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
41,5,566453537.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,447,30000025.00000,0.00001,Private Sector_0,0,Private Sector,1,1,1,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2797,448,0.00000,0.00000,Private Sector_0,0,Private Sector,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2799,449,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2806,450,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan


In [139]:
data_ind_U.shape

(346, 42)

In [140]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [141]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [142]:
# pyodbc.drivers()
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock) where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Individual'", conn)
reqd_cols_per = rule_df['RULE_NO'].to_list()
len(reqd_cols_per)

35

In [143]:
col_with_data = data_ind_U.columns.to_list()

In [144]:
col_with_data = [col for col in col_with_data if 'P0' in col]

In [145]:
col_with_data

['P002001001',
 'P002001002',
 'P002001004',
 'P002001005',
 'P002001007',
 'P002001008',
 'P002001009',
 'P002001010',
 'P002001011',
 'P002001012',
 'P002001015',
 'P002001018',
 'P002001021',
 'P002001024',
 'P002001025',
 'P002001027',
 'P002001028',
 'P002001030',
 'P002001031',
 'P002001033',
 'P002001034',
 'P002003001',
 'P002003002',
 'P002003003',
 'P002003005',
 'P002003006',
 'P002003008',
 'P002001016',
 'P002001029',
 'P002001019',
 'P003001002',
 'P002001032',
 'P002001023',
 'P002003007',
 'P002001026']

In [146]:
reqd_cols_per

['P002001031',
 'P002001032',
 'P002001033',
 'P002001015',
 'P002001001',
 'P002001002',
 'P002001004',
 'P002001005',
 'P002001007',
 'P002001008',
 'P002001009',
 'P002001010',
 'P002001011',
 'P002001012',
 'P002001026',
 'P002001016',
 'P002001018',
 'P002001019',
 'P002001021',
 'P002001023',
 'P002001024',
 'P002001025',
 'P002001027',
 'P002001030',
 'P002001034',
 'P002003001',
 'P002003002',
 'P002003003',
 'P002003005',
 'P002003006',
 'P002003007',
 'P002003008',
 'P003001002',
 'P002001028',
 'P002001029']

In [147]:
col_without_data = set(reqd_cols_per) - set(col_with_data)

In [148]:
col_without_data

set()

In [149]:
for col in col_without_data:
    data_ind_U[col] = np.nan

In [150]:
data_ind_U

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE,P002001016,P002001029,P002001019,P003001002,P002001032,P002001023,P002003007,P002001026
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
11,2,566453534.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
21,3,565000000.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
31,4,566453536.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
41,5,566453537.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,447,30000025.00000,0.00001,Private Sector_0,0,Private Sector,1,1,1,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2797,448,0.00000,0.00000,Private Sector_0,0,Private Sector,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2799,449,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2806,450,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan


In [151]:
ind_data_thresholds_U.to_csv(r"/data/threshold_individual_U.csv",index=False)
#Uncomment to run on org
# org_data_thresholds_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv",index=False)

data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U.csv",index=False)

weight_table_ind_U.to_csv(r"/data/weight_individual_U.csv",index=False)
# weight_table_org_U.to_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv",index=False)


In [152]:
data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U.csv",index=False)
data_ind_U.head(5)

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE,P002001016,P002001029,P002001019,P003001002,P002001032,P002001023,P002003007,P002001026
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
11,2,566453534.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
21,3,565000000.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
31,4,566453536.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
41,5,566453537.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan


In [153]:
data_ind_U

,PERSON_MASTER_ID,PEER_GROUP_AMNT,PEER_GROUP_AMNT_Scaled,PEER_GROUP_ID,Cluster_labels,OCCUPATION_BUSINESS_CLEAN,P002001001,P002001002,P002001004,P002001005,P002001007,P002001008,P002001009,P002001010,P002001011,P002001012,P002001015,P002001018,P002001021,P002001024,P002001025,P002001027,P002001028,P002001030,P002001031,P002001033,P002001034,P002003001,P002003002,P002003003,P002003005,P002003006,P002003008,CREATE_DATE,P002001016,P002001029,P002001019,P003001002,P002001032,P002001023,P002003007,P002001026
0,1,2264360596.00000,0.00047,Government Sector_0,0,Government Sector,4,1,4,NaN,4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
11,2,566453534.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
21,3,565000000.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
31,4,566453536.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
41,5,566453537.00000,0.00012,Government Sector_0,0,Government Sector,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2789,447,30000025.00000,0.00001,Private Sector_0,0,Private Sector,1,1,1,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2797,448,0.00000,0.00000,Private Sector_0,0,Private Sector,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2799,449,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan
2806,450,1200000.00000,0.00000,Private Sector_0,0,Private Sector,1,1,1,1,1,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-21,nan,nan,nan,nan,nan,nan,nan,nan


In [154]:

# a = data_ind_U[data_ind_U['PEER_GROUP_ID'] == 'Business_0']['P002001007']
# a = a.dropna()
# a = a.astype('int32')
# print(a.describe())
# print("median",np.median(a))
# print("mode",statistics.mode(a))